In [5]:

import numpy as np
import pandas as pd

N = 20
ts = pd.Series(
    np.arange(N),
    index=pd.period_range(start='2010', freq='A', periods=N, name='Year'),
    dtype=pd.Int8Dtype,
)

print("ts: \n", ts)

# Lag features
X = pd.DataFrame({
    'y_lag_2': ts.shift(2),
    'y_lag_3': ts.shift(3),
    'y_lag_4': ts.shift(4),
    'y_lag_5': ts.shift(5),
    'y_lag_6': ts.shift(6),    
})

print("X: \n",X)

# Multistep targets
y = pd.DataFrame({
    'y_step_3': ts.shift(-2),
    'y_step_2': ts.shift(-1),
    'y_step_1': ts,
})

print("y: \n", y)

data = pd.concat({'Targets': y, 'Features': X}, axis=1)

data.head(10).style.set_properties(['Targets'], **{'background-color': 'LavenderBlush'}) \
                   .set_properties(['Features'], **{'background-color': 'Lavender'})

X: 
      y_lag_2 y_lag_3 y_lag_4 y_lag_5 y_lag_6
Year                                        
2010     NaN     NaN     NaN     NaN     NaN
2011     NaN     NaN     NaN     NaN     NaN
2012       0     NaN     NaN     NaN     NaN
2013       1       0     NaN     NaN     NaN
2014       2       1       0     NaN     NaN
2015       3       2       1       0     NaN
2016       4       3       2       1       0
2017       5       4       3       2       1
2018       6       5       4       3       2
2019       7       6       5       4       3
2020       8       7       6       5       4
2021       9       8       7       6       5
2022      10       9       8       7       6
2023      11      10       9       8       7
2024      12      11      10       9       8
2025      13      12      11      10       9
2026      14      13      12      11      10
2027      15      14      13      12      11
2028      16      15      14      13      12
2029      17      16      15      14      13
y: 
 